<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - SH openEO backend NDVI use case
### Calculate mean NDVI over time (as synchronous and batch job) and export the result (locally, via signed URL)

In [ ]:
import openeo
import rioxarray

**1. Open the connect to openEO Platform (development backend) and authenticate**  
- establish connection to the openEO Platform back-end
- connection object is your central gateway to load data

In [ ]:
from openeo.rest.auth.config import RefreshTokenStore
RefreshTokenStore().remove()

In [ ]:
# Connect to openEO Platform  back-end (https://openeo.cloud) which has SH production backend implemented
# https://openeo.sentinel-hub.com/production/
connection = openeo.connect("https://openeo.sentinel-hub.com/production/")

# Authenticate via EGI
connection.authenticate_oidc()

**2. Define spatial and temporal extend, bands and specify backend to be used**

In [ ]:
# Load data cube from Sentinel2_L2A collection for a 3-weeks cloud free priod in July 2021 north of Ljubljana
# (5 aquisitions)
cube = connection.load_collection(
    "SENTINEL2_L2A_SENTINELHUB",
    spatial_extent={"west": 14.583040689005495, "south": 46.10348118219517, "east": 14.599965321685639, 
                    "north": 46.11226598409823},
    temporal_extent=["2021-07-09", "2022-07-30"],
    bands=["B04", "B08"], properties={"provider:backend":lambda v: v =="sentinelhub"}
)

# The "provider:backend" can be used for collections that are supported by multiple backends to 
# specify on which backend the process graph should be run. For SENTINEL2_L2A_SENTINELHUB available options are "Vito"
# and "sentinelhub"

**3. Calculate the mean over the temporal extend and afterwards calculate the NDVI**

In [ ]:
# Calculate mean of B04 and B08
cube_mean = cube.reduce_dimension(dimension="t", reducer="mean")

In [ ]:
# Calculate NDVI
cube_mean_ndvi_all_bands = cube_mean.ndvi(nir = "B08", red = "B04", target_band = "NDVI")

In [ ]:
# Filter bands
cube_mean_ndvi = cube_mean_ndvi_all_bands.filter_bands(["NDVI"])

## Synchronous processing
**4. Download data synchronously**

In [ ]:
# depending on the backend load this synchronous call sometimes times out
# as the result is part of the repository you can continue with the next cell 
# and visualize the result and run the job as batch job
cube_mean_ndvi.download("cube_mean_ndvi.tiff")

**5. Plot NDVI data on map**

In [ ]:
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio("./cube_mean_ndvi.tiff")

# Covert our xarray.DataArray into a xarray.Dataset
geotiff_ds = geotiff_da.to_dataset('band').rename({1: 'NDVI'})

In [ ]:
# plot mean NDVI data on map
geotiff_ds.NDVI.plot()

## Batch processing
**4. Save results as Geotiff**

In [ ]:
# Save results
cube_mean_ndvi_tiff = cube_mean_ndvi.save_result(format="GTiff")

**5. Create a batch job and start it.**

More info about batch jobs can be found here:

- https://openeo.org/documentation/1.0/glossary.html#data-processing-modes

- https://open-eo.github.io/openeo-python-client/batch_jobs.html

In [ ]:
# create the job 
job = cube_mean_ndvi_tiff.create_job(title = 'Mean_NDVI_Ljubljana_demo')

# save job id and print it
job_id = job.job_id
print("Batch job created with id: ",job_id)

In [ ]:
# start the job
job.start_job() 

In [ ]:
# output batch job status 
print("Batch job with id: ",job_id, ' is ',job.status())

In [ ]:
# get job metadata to get signed URL from Assets
# for jobs on SH production use 00152be2-2824-49d6-b3c8-0c3bb2f3013c
job = connection.job('00152be2-2824-49d6-b3c8-0c3bb2f3013c')
job.get_results()